# **Experimento com RNN: Verificar saídas e estados do LSTM e GRU para diferentes argumentos de entrada**

# **Recurrent Neural Networks (RNN) applied to Natural Language Processing (NLP)**

- Neste projeto, vamos **avaliar os efeitos relacionados à utilização de diferentes argumentos ao se invocar o GRU ou LSTM**.
- Em particular, podemos passar como argumento `return_sequences = True`, que fará o sistema retornar as saídas de cada ponto no tempo, ao invés de apenas a última saída.
- Também podemos passar `return_states = True`, o que fará o sistema retornar os estados das camadas ocultas (hidden states) em adição à saída da rede.
- Ao se trabalhar com uma API Machine Learning, uma das coisas mais importantes é compreender o formato de cada uma das entradas e saídas.

Neste projeto, **vamos provar que, dependendo da unidade recursiva utilizada, o hidden state será exatamente igual à saída** (`hidden state = output`).

# **Importar bibliotecas para análise**

In [ ]:
# https://deeplearningcourses.com/c/deep-learning-advanced-nlp
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

from keras.models import Model
from keras.layers import Input, LSTM, GRU
import numpy as np
import matplotlib.pyplot as plt

try:
  import keras.backend as K
  if len(K.tensorflow_backend._get_available_gpus()) > 0:
    from keras.layers import CuDNNLSTM as LSTM
    from keras.layers import CuDNNGRU as GRU
except:
  pass

# **Configurações de parâmetros de forma dos dados**

```
- T = sequence length (number of words);
- D = input dimensionality (size of the word vectors);
- M = hidden layer size;
- K = number of output classes.
```



In [ ]:
T = 8
D = 2
M = 3

Vamos criar também dados de entrada aleatórios.
- Esta amostra será uma matriz de números aleatórios entre -1 a 1, de dimensão T x D, criada com a correspondente função do NumPy.

In [ ]:
X = np.random.randn(1, T, D)
print(X)

[[[ 0.17128711  0.02895866]
  [ 0.08442127 -1.4335657 ]
  [-0.50074148 -0.91084879]
  [ 1.07310412  0.13638003]
  [-1.39514574 -1.14349911]
  [ 0.21751521  0.67588334]
  [-0.9521634  -1.00280792]
  [-0.14377812 -0.99916313]]]


Esta matriz poderia ser a **representação de uma única sequência de vetores de palavras, ou poderia ser algum outro sinal obtido**.

# **Primeiro experimento LSTM: Função lstm1** 

- A primeira coisa que fazemos aqui é criar nossa camada de input (input layer), de dimensão T x D, e passamos os dados, a seguir, pela camada LSTM.
- Na criação da camada LSTM, passamos o argumento `return_state = True`.
- Lembre-se que temos, no LSTM, dois estados, h (hidden) e c (cell). 
- Quando criamos um modelo que mais tarde realizará previsões (ou seja, que retornará a saída do LSTM), precisamos ser capazes de capturar 3 saídas diferentes:

  1) A saída de fato do LSTM;
  
  2) Hidden state (h); e
  
  3) Cell state (c).

In [ ]:
def lstm1():
  input_ = Input(shape=(T, D))
  rnn = LSTM(M, return_state=True)
  x = rnn(input_)

  model = Model(inputs=input_, outputs=x)
  o, h, c = model.predict(X)
  print("o:", o)
  print("h:", h)
  print("c:", c)

# **Segundo experimento LSTM: Função lstm2** 

- A função é igual à lstm1. A única diferença é que também passamos o argumento:

```
return_sequences=True
```
- O objetivo deste experimento é verificar quais saídas serão retornadas, e como estas saídas se relacionam ao que ocorre quando o parâmetro está em seu valor-padrão, `False`.

In [ ]:
def lstm2():
  input_ = Input(shape=(T, D))
  rnn = LSTM(M, return_state=True, return_sequences=True)
  # rnn = GRU(M, return_state=True)
  x = rnn(input_)

  model = Model(inputs=input_, outputs=x)
  o, h, c = model.predict(X)
  print("o:", o)
  print("h:", h)
  print("c:", c)

# **Primeiro experimento GRU: Função GRU1** 

- Esta função espelha a função lstm1, quando substituímos LSTM por GRU.
- Lembre-se que o GRU possui apenas 1 estado, h.
- Portanto, quando evocamos a previsão do modelo (`model.predict(X)`), **são retornados apenas 2 valores ao invés dos 3 valores de saída retornados pelo LSTM**.

In [ ]:
def gru1():
  input_ = Input(shape=(T, D))
  rnn = GRU(M, return_state=True)
  x = rnn(input_)

  model = Model(inputs=input_, outputs=x)
  o, h = model.predict(X)
  print("o:", o)
  print("h:", h)

# **Segundo experimento GRU: Função GRU2** 

- Esta função espelha a função lstm2, quando substituímos LSTM por GRU.
- Assim, aqui temos `return_state=True` e `return_sequences=True`.
- Mais uma vez, como o GRU possui apenas 1 estado (h), são retornados apenas 2 valores.

In [ ]:
def gru2():
  input_ = Input(shape=(T, D))
  rnn = GRU(M, return_state=True, return_sequences=True)
  x = rnn(input_)

  model = Model(inputs=input_, outputs=x)
  o, h = model.predict(X)
  print("o:", o)
  print("h:", h)

Note que o comando `model.predict(X)` calcula os valores previstos pelo modelo `model` para cada um dos valores do dataframe `X` fornecido como input.

In [ ]:
print("lstm1:")
lstm1()

lstm1:
o: [[-0.17164306 -0.14928916 -0.18100695]]
h: [[-0.17164306 -0.14928916 -0.18100695]]
c: [[-0.46857202 -0.25278148 -0.50404453]]


Aqui no lstm1 nós retornamos os estados, mas nenhuma sequência.
- Note que **a saída o ("output") é exatamente igual a h ("hidden state)**.
- Portanto, para o LSTM, h = o, o que desejávamos verificar.

In [ ]:
print("lstm2:")
lstm2()

lstm2:
o: [[[ 0.02380813 -0.02079004 -0.01417855]
  [-0.05204503  0.09870689 -0.05733851]
  [-0.10372683  0.16380192 -0.05819457]
  [ 0.1137677  -0.06495867 -0.1178459 ]
  [-0.04452203  0.03250882  0.00755201]
  [ 0.02075053 -0.07436121  0.01028471]
  [-0.07959417  0.0269038   0.06434048]
  [-0.11766418  0.12295073  0.02959999]]]
h: [[-0.11766418  0.12295073  0.02959999]]
c: [[-0.22961718  0.19231719  0.0732884 ]]


- Veja que h e c são exatamente iguais no lstm1 e lstm2.
- Porém, a saída o é maior em lstm2, função na qual passamos como argumento o comando de retornar as sequências.
- Note que o é formado por uma sequência de 8 arrays. Isto ocorre porque, lembrando das definições de parâmetros no começo do notebook: `T = 8`.
- Repare ainda que o último elemento de o é igual ao hidden state h. Quando definimos `return_sequences=False (lstm1)`, apenas este último array é retornado em o.
- Assim, **h e c representam, na realidade, os hidden states finais do LSTM**.
- Em outras palavras, **h e c são o hidden state e o cell state no último passo temporal do input**.

In [ ]:
print("gru1:")
gru1()

gru1:
o: [[-0.41285896  0.13199161 -0.19100542]]
h: [[-0.41285896  0.13199161 -0.19100542]]


Aqui a observação é a mesma feita para o lstm1: como a saída o contém apenas o último estágio temporal (valor final), o ("output") e h ("hidden state") são iguais.

In [ ]:
print("gru2:")
gru2()

gru2:
o: [[[ 0.05330895  0.05650215  0.02914332]
  [ 0.0646524   0.09152526 -0.11673455]
  [-0.13698067 -0.02340787 -0.20172621]
  [ 0.16627601  0.3503932   0.18803528]
  [-0.3561732   0.06852631 -0.03838757]
  [-0.12937838  0.06368127  0.10767905]
  [-0.37870485 -0.11547271 -0.08032594]
  [-0.19312723 -0.11588701 -0.11327047]]]
h: [[-0.19312723 -0.11588701 -0.11327047]]


Assim como no lstm2, novamente temos uma saída o formada por 8 arrays, consequência da definição `T = 8`. Além disso, o último elemento de o é igual ao hidden state h. Quando definimos `return_sequences=False (gru1)`, apenas este último array é retornado em o.

# **Conclusão**

- O objetivo deste notebook é ilustrar as diferentes saídas ("output", o) e estados ("hidden state", h, e "cell state", c) retornados pelo sistema quando definimos `return_state = True` ou `return_sequences = True`.
- Em notebooks mais complexos sobre RNN, volte aqui para recordar as saídas e estados esperados, bem como as respectivas interpretações.
- Uma técnica útil para o "debug" da rede neural é verificar se tudo está no formato/dimensão correta a cada etapa de cálculo.
- Muitas vezes, **perdemos esta granularidade ao utilizar operações de agregação como Pooling ou soma das médias**. Este caso é particularmente difícil de ser corrigido, pois o sistema retorna um valor incorreto ao invés de uma mensagem de erro, podendo passar a impressão que o código está correto quando não o está.